In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import datetime
from datetime import timedelta
import matplotlib as mt

## Create Function to Import Data from Yahoo Finance

In [2]:
def get_yfinance_data(ticker_name, period):
    ticker = ticker_name
    period = period
    
    data = yf.Ticker(ticker)
    historical_close = pd.DataFrame(data.history(period = period))["Close"]
    return(historical_close)

In [3]:
yahoo_data_list = ["SPY", "DJI"]

In [4]:
for ticker in yahoo_data_list:
    data = pd.DataFrame(get_yfinance_data(ticker, "max"))

## Read CSV Files in the Folder

In [5]:
# Adding working directory
directory = os.listdir("C:/Users/peter/Desktop/python-recession-model")

In [6]:
csv_file_list = []
csv_name = []
for file in directory:
    if file.endswith(".csv"):
        data_name = file.split(".")[0]
        data = pd.read_csv(file)
        csv_name.append(data_name)
        csv_file_list.append(data)

In [7]:
csv_name1 = pd.DataFrame(csv_name).apply(lambda x: x.str.lower().str.replace('(', '').str.replace(')', '').str.replace('-', '').str.replace(',', ' ').str.replace(' ', '_').str.replace('__', '_'))

C:\Users\peter\AppData\Local\Temp\ipykernel_19264\4020979771.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  csv_name1 = pd.DataFrame(csv_name).apply(lambda x: x.str.lower().str.replace('(', '').str.replace(')', '').str.replace('-', '').str.replace(',', ' ').str.replace(' ', '_').str.replace('__', '_'))


In [8]:
table_dict = {}

for i in range(30):
        table_name = csv_name1.at[i, 0]
        data = pd.DataFrame(csv_file_list[i])
        table_dict[table_name] = data

In [9]:
# Format the date column name
for keys, values in table_dict.items():
    table_dict[keys] = values.rename(str.lower, axis = "columns")

In [10]:
table_count = 0
for keys, values in table_dict.items():
    number_row = len(values)
    top_two = values.head(2)
    bottom_two = values.tail(2)
    table_count = table_count + 1
    print("Table Number: " + str(table_count))
    print("Table Name: " + keys)
    print("Number of Row: "  + str(number_row))
    print(top_two)
    print(bottom_two)
    print("____________________________________________________________________________________________")

Table Number: 1
Table Name: capacity_utilization_total_index
Number of Row: 681
         date      tcu
0  1967-01-01  89.3902
1  1967-02-01  87.9750
           date      tcu
679  2023-08-01  79.5385
680  2023-09-01  79.6786
____________________________________________________________________________________________
Table Number: 2
Table Name: consumer_price_index_for_all_urban_consumers_all_items_in_u
Number of Row: 921
         date  cpiaucsl
0  1947-01-01     21.48
1  1947-02-01     21.62
           date  cpiaucsl
919  2023-08-01   306.269
920  2023-09-01   307.481
____________________________________________________________________________________________
Table Number: 3
Table Name: cpi
Number of Row: 914
         date  cpiaucsl
0  1947-01-01     21.48
1  1947-02-01     21.62
           date  cpiaucsl
912  2023-01-01   300.536
913  2023-02-01   301.648
____________________________________________________________________________________________
Table Number: 4
Table Name: crude_oil_p

# Format Date Object

In [11]:
def guess_date(string):
    for date_format in ["%Y-%m-%d", "%m/%d/%Y", "%d-%b-%y"]:
        try:
            return datetime.strptime(string, date_format).date()
        except ValueError:
            continue
    raise ValueError(string)

In [12]:
for keys, values in table_dict.items():
    date_string = values["date"]
    date_format = ["%Y-%m-%d", "%m/%d/%Y", "%d-%b-%Y"]
    values["date"] = date_string.apply(lambda x: guess_date(x)).apply(lambda x: x.replace(year = x.year - 100) if x.year > 2030 else x )

In [13]:
# Edge Case Fed Model Date Handling
delta = timedelta(days = 1)
table_dict["fed_model_data"]["date"] = table_dict["fed_model_data"]["date"].apply(lambda x: x + delta)

## Basic Information of each dataset

In [14]:
# Function to provide basic information about a list of dataset
def data_basic_infor(table_dict):
    table_count = 0
    for keys, values in table_dict.items():
        number_row = len(values)
        start_date = min(values["date"])
        end_date = max(values["date"])
        table_count = table_count + 1
        column_name = values.columns
        frequency = values["date"][1] - values["date"][0]
        print("Table Number: " + str(table_count))
        print("Table Name: " + keys)
        print("Column Names: " + str(column_name[0]) + ", " + str(column_name[1]))
        print("Number of Row: "  + str(number_row))
        print("Frequency: " + str(frequency))
        print("Start Date: " + str(start_date))
        print("End Date: " + str(end_date))
        print("____________________________________________________________________________________________")

In [15]:
# Overview of basic information about each dataset
data_basic_infor(table_dict)

Table Number: 1
Table Name: capacity_utilization_total_index
Column Names: date, tcu
Number of Row: 681
Frequency: 31 days, 0:00:00
Start Date: 1967-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 2
Table Name: consumer_price_index_for_all_urban_consumers_all_items_in_u
Column Names: date, cpiaucsl
Number of Row: 921
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 3
Table Name: cpi
Column Names: date, cpiaucsl
Number of Row: 914
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-02-01
____________________________________________________________________________________________
Table Number: 4
Table Name: crude_oil_prices_west_texas_intermediate_wti_cushing_oklahoma
Column Names: date, dcoilwtico
Number of Row: 9863
Frequency: 1 day, 0:00:00
Start Date: 1986-0

## Filter the Data and Create Updated List of Dataset

In [16]:
# Filter the list to delete based on the following requirements:
# 1. Has start date before 1975
# 2. Has end date after 2023
# 3. The update frequency is less than a month

list_to_delete = []
delta = timedelta(days = 32)
for keys, values in table_dict.items():
    frequency = values["date"][1] - values["date"][0]
    if min(values["date"]).year > 1975:
        list_to_delete.append(keys)
    elif frequency > delta:
        list_to_delete.append(keys)
    elif max(values["date"]).year < 2023:
        list_to_delete.append(keys)
        
        
print("The delete list has " + str(len(list_to_delete)) + " and has following items: ")
print("")
for table_name in list_to_delete:
    print(table_name)

The delete list has 10 and has following items: 

crude_oil_prices_west_texas_intermediate_wti_cushing_oklahoma
dji
economic_policy_uncertainty_index_for_united_states
inflation_consumer_prices_for_the_united_states
leading_index_for_the_united_states
t10y2y_daily
t10y3m_daily
university_of_michigan_inflation_expectation
velocity_of_m2_money_stock
vix


In [17]:
updated_table_dict = table_dict

for i in list_to_delete:
    del updated_table_dict[i]


In [18]:
data_basic_infor(updated_table_dict)

Table Number: 1
Table Name: capacity_utilization_total_index
Column Names: date, tcu
Number of Row: 681
Frequency: 31 days, 0:00:00
Start Date: 1967-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 2
Table Name: consumer_price_index_for_all_urban_consumers_all_items_in_u
Column Names: date, cpiaucsl
Number of Row: 921
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 3
Table Name: cpi
Column Names: date, cpiaucsl
Number of Row: 914
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-02-01
____________________________________________________________________________________________
Table Number: 4
Table Name: federal_funds
Column Names: date, fedfunds
Number of Row: 828
Frequency: 31 days, 0:00:00
Start Date: 1954-07-01
End Date: 2023-06-01
_______________________

# Merge for final table

In [19]:
final_data = updated_table_dict["capacity_utilization_total_index"]

for keys, values in updated_table_dict.items():
    print(keys)
    final_data = final_data.merge(values, how = "left", on = "date")

capacity_utilization_total_index
consumer_price_index_for_all_urban_consumers_all_items_in_u
cpi
federal_funds
federal_funds_effective_rate_daily
fed_model_data
industrial_production_total_index
initial_claims
labor_force_participation_rate
m1_monthly
m2_monthly
m3_monthly
market_yield_on_u
monetary_aggregates_and_their_components_broad_money_and_components_m3_for_united_states
monetary_base_total
nber_based_recession_indicators_for_the_united_states_from_the_period_following_the_peak_through_the_trough
sp500
unemployment_rate
university_of_michigan_consumer_sentiment


In [20]:
final_data.isna().sum()

date                                                0
tcu_x                                               0
tcu_y                                               0
cpiaucsl_x                                          0
cpiaucsl_y                                          7
fedfunds                                            3
dff                                                 0
10 year treasury yield                             14
3 month treasury yield                             14
3 month treasury yield (bond equivalent basis)     14
spread                                             14
rec_prob                                           14
nber_rec                                           14
indpro                                              0
icsa                                              583
civpart                                             3
m1sl                                                5
m2sl                                                5
mabmm301usm189s_x           

In [21]:
final_data

,date,tcu_x,tcu_y,cpiaucsl_x,cpiaucsl_y,fedfunds,dff,10 year treasury yield,3 month treasury yield,3 month treasury yield (bond equivalent basis),...,m1sl,m2sl,mabmm301usm189s_x,dgs10,mabmm301usm189s_y,bogmbase,usrec,value,unrate,umcsent
0,1967-01-01,89.3902,89.3902,32.900,32.9,4.94,5.00,4.84,4.96,5.09,...,171.9,481.6,4.816000e+11,NaN,4.816000e+11,63800.0,0,84.45,3.9,.
1,1967-02-01,87.9750,87.9750,33.000,33.0,5.00,4.00,4.58,4.72,4.84,...,173.0,485.1,4.851000e+11,4.52,4.851000e+11,63300.0,0,87.36,3.8,94.1
2,1967-03-01,87.0835,87.0835,33.000,33.0,4.53,3.75,4.63,4.56,4.68,...,174.8,489.7,4.897000e+11,4.66,4.897000e+11,63200.0,0,89.42,3.8,.
3,1967-04-01,87.5089,87.5089,33.100,33.1,4.05,4.50,4.54,4.26,4.37,...,174.2,492.1,4.921000e+11,NaN,4.921000e+11,63300.0,0,90.96,3.8,.
4,1967-05-01,86.3566,86.3566,33.100,33.1,3.94,4.00,4.59,3.84,3.93,...,175.7,497.2,4.972000e+11,4.77,4.972000e+11,63400.0,0,92.59,3.8,95.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,2023-05-01,79.4570,79.4570,303.294,NaN,5.06,4.83,3.46,4.92,5.05,...,NaN,NaN,NaN,3.59,2.084170e+13,5569600.0,0,NaN,3.7,59.2
677,2023-06-01,78.9301,78.9301,303.841,NaN,5.08,5.08,3.57,5.14,5.28,...,NaN,NaN,NaN,3.61,2.089040e+13,5608500.0,0,NaN,3.6,64.4
678,2023-07-01,79.6066,79.6066,304.348,NaN,NaN,5.08,3.75,5.16,5.30,...,NaN,NaN,NaN,NaN,2.090530e+13,5517800.0,0,NaN,3.5,71.6
679,2023-08-01,79.5385,79.5385,306.269,NaN,NaN,5.33,3.90,5.25,5.39,...,NaN,NaN,NaN,4.05,2.086530e+13,5559100.0,0,NaN,3.8,69.5
